# Projeto Final - Machine Learning

## Alunos:
- Guilherme Fontana Louro
- Pedro Altobelli Teixeira Pinto

## Objetivo
- Criar um modelo de Machine Learning capaz de sugerir uma boa aposta para um jogo de futebol. Essa aposta pode ser o resultado final do jogo, número de gols, cartões, escanteios, etc.

## Dataset
- O dataset utilizado foi o [Football Database](https://www.kaggle.com/datasets/technika148/football-database), disponível no Kaggle. Ele contém dados de jogos de futebol de 11 ligas europeias, desde a temporada 2008/2009 até a 2019/2020. Os dados foram coletados do site [football-data.co.uk](https://www.football-data.co.uk/).

# Análise Exploratória inicial

In [1]:
import pathlib
filename = pathlib.Path.cwd() / '../data/teams.csv'

In [2]:
import pandas as pd
data = pd.read_csv(filename)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   teamID  146 non-null    int64 
 1   name    146 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.4+ KB


In [4]:
data.head(5)

,teamID,name
0,71,Aston Villa
1,72,Everton
2,74,Southampton
3,75,Leicester
4,76,West Bromwich Albion
